In [1]:

%pprint
%matplotlib inline
import sys
import os.path as osp
import os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join(os.pardir, os.pardir, 'share')); assert osp.exists(shared_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)

# Import needed libraries
from bs4 import BeautifulSoup as bs
from pandas import DataFrame
import ast

Pretty printing has been turned OFF


In [7]:

def find_self_references_and_definitions(file_path):
    """
    Analyze a file to find all references to `self.` attributes and methods,
    and check if they are defined.

    Args:
        file_path (str): Path to the Python file to analyze.

    Returns:
        dict: A dictionary with class names as keys and a report of all `self.` attributes
              and methods (whether they are defined or missing) for each class.
    """
    # Read the source code
    with open(file_path, "r") as f:
        source_code = f.read()

    # Parse the source code into an AST
    tree = ast.parse(source_code)

    # Store results
    results = {}

    # Traverse the AST to find class definitions
    for node in tree.body:
        if isinstance(node, ast.ClassDef):  # Found a class
            class_name = node.name
            # Collect all `self.<attribute>` references and definitions
            self_references = set()
            self_definitions = set()

            # Collect all method names in the class (to treat as `self.<method>` definitions)
            method_names = set()

            # Traverse the class body
            for class_node in node.body:
                # If it's a function (method), record its name
                if isinstance(class_node, ast.FunctionDef):
                    method_names.add(class_node.name)

                    # Walk through the method's body to find self references and assignments
                    for stmt in ast.walk(class_node):
                        # Look for `self.<attribute>` references
                        if (
                            isinstance(stmt, ast.Attribute)
                            and isinstance(stmt.value, ast.Name)
                            and stmt.value.id == "self"
                        ):
                            self_references.add(stmt.attr)

                        # Look for `self.<attribute> = ...` assignments
                        if isinstance(stmt, ast.Assign):
                            for target in stmt.targets:
                                if (
                                    isinstance(target, ast.Attribute)
                                    and isinstance(target.value, ast.Name)
                                    and target.value.id == "self"
                                ):
                                    self_definitions.add(target.attr)

            # Add method names as definitions (since they are callable via `self.<method>`)
            self_definitions.update(method_names)

            # Determine missing definitions
            missing_definitions = self_references - self_definitions

            # Save results for this class
            results[class_name] = {
                "self_references": self_references,
                "self_definitions": self_definitions,
                "missing_definitions": missing_definitions,
            }

    return results

In [3]:

def analyze_modules_in_folder(folder_path):
    """
    Analyze all Python files in a folder for `self.` references and definitions.

    Args:
        folder_path (str): Path to the folder containing Python files to analyze.

    Returns:
        dict: A dictionary with file names as keys and class analysis results as values.
    """
    analysis_results = {}

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".py"):  # Only process Python files
            file_path = os.path.join(folder_path, file_name)
            results = find_self_references_and_definitions(file_path)
            analysis_results[file_name] = results

    return analysis_results

In [4]:

def print_analysis_results(analysis_results):
    """
    Pretty-print the analysis results.

    Args:
        analysis_results (dict): The results from analyzing modules.
    """
    for file_name, classes in analysis_results.items():
        print(f"File: {file_name}")
        for class_name, data in classes.items():
            print(f"  Class: {class_name}")
            print(f"    - self references: {data['self_references']}")
            print(f"    - self definitions: {data['self_definitions']}")
            print(f"    - missing definitions: {data['missing_definitions']}")
        print()

In [8]:

# Folder containing the Python modules to analyze
folder_path = osp.abspath(osp.join(os.pardir, os.pardir, 'share'))

# Analyze all modules in the folder
results = analyze_modules_in_folder(folder_path)

# Print the results
print_analysis_results(results)

File: data_analysis.py
  Class: DataAnalysis
    - self references: {'get_euclidean_distance', 'facebook_aspect_ratio', 'get_inf_nan_mask', 'get_flattened_dictionary', 'lower_ascii_regex', 'get_r_squared_value_latex'}
    - self definitions: {'get_nearest_neighbor', 'plot_histogram', 'open_path_in_notepad', 'get_numeric_columns', 'get_r_squared_value_latex', 'first_order_linear_scatterplot', 'get_jitter_list', 'get_regexed_dataframe', 'plot_line_with_error_bars', 'modalize_columns', 'get_euclidean_distance', 'get_regexed_columns', 'get_column_descriptions', 'get_spearman_rho_value_latex', 'get_flattened_dictionary', 'get_minority_combinations', 'get_wiki_infobox_data_frame', 'one_hot_encode', 'count_swaps_to_perfect_order', 'convert_strings_to_integers'}
    - missing definitions: {'get_inf_nan_mask', 'facebook_aspect_ratio', 'lower_ascii_regex'}

File: data_preparation.py
  Class: DataPreparation
    - self references: {'object_evaluators', 'github_folder', 'get_page_soup', 'encoding_


----